# Summary of the notation of the paper and its translation into code

# Sets
$G=(V, A)$ is the graph, where $V=\{1, 2, \dots, N\}$. Arcs are directional, where arc $(i,j)$ leads from node $i$ to node $j$ (and vice versa). All arcs can be travelled in both directions.

The journey of one train, its path, is denoted as $P=\{1, i_1, i_2, \dots, N\}$, so that $1$ is the origin of the path, $N$ its destination, and $i_1, i_2, \dots$ are the nodes travelled. A feasible path is one where all movement constraints and time constraints are fulfilled.


# Parameters and Variables

$f(i,j)$: according to the authors, it is the earliest feasible arrival time at node $i$, provided one can depart along arc $(i,j)$. For now, a better interpretation seems to be that $f(i,j)$ is the arrival time at node $i$ under the current solution for a given path. Written as `f[i][j]`

# Constraints
Arc $a=(i,j)$ is closed for all other trains when a train travels along it from station $i$ to statoin $j$, departing at time $t$ and traveling for duration $d(a)$ . This closing period is denoted as

$$[t, t+d(a)]$$

A station is closed for a security duration $\Delta$ after a train leaves a 
station.

A station has separate capacities for parking and passing through. Only one train can be parked at a station at each moment, but on other train can pass through the station while one other train is parked there.

A route along arc $(i,j)$ is blocked if, for this specific route, the train can not depart from mode $i$ to node $j$. This can have different reasons, which will be explained below.

# Splitting the problem



## Calculating arrival times

The problem can be solved by finding the earliest arrival time at each node, beginning from the start node. Hence, the main task is to calculate an arrival time for a current solution, then updating it if later restrictions make that necessary.

Once the arrival time $f(i,j)$ is known, there are three different possibilities for the arrival time $f(j,k)$:

1. The path is not blocked along arc $(j,k)$. That means that the earliest arrival time $f(j,k)$ is the arrival time $f(i,j)$, plus waiting time at node $i$ and travel time on arc $(i,j)$, for the node $i$ that minimises this sum.
\begin{align*}
f(j,k) = \underset{i,p_i}{\min}[f(i,j) + p_i + t_{ij}]
\end{align*}
We have to account for the possibility that an arc may be reached via different routes.

2. The path is blocked along arc $(j,k)$ and can not be unblocked. In that case, the route can not contain this arc. A route is irreversibly blocked if the train arrives after the closing of the departure time window.

3. A path is blocked along arc $(i,j)$, but can be unblocked. In this case, the train might arrive within either the parking time window of departure time window by extending its parking time at other nodes or taking another route. The last option is not considered by the authors. The first option means that we have now two possible values for $f(i_1, i_2)$: to the already existing solution derived from earlier steps, we add the new one with increased parking times in some node preceding $i_1$.

If an arc is blocked for a current path solution, we write $f(i,j) = \infty$.

## Blocked routes

A path is blocked on arc $(i,j)$ if 

1. Departure is allowed before parking (it is not necessary to wait for departure while parking), but arrival occurs before departure is allowed: $f(i,j) < \gamma_{ij} < \alpha_i$
2. Departure is allowed at some point in time during the parking period (it is possible to wait for departure while parking), but arrival occurs before parking is allowed: $f(i,j) < \alpha_i < \gamma_{ij} < \beta_i$
3. Departure is allowed after the parking period (it is not possible to wait for departure while parking), and arrival occurs before parking: $f(i,j) < \alpha_i < \beta_i < \gamma_{ij}$
4. Arrival occurs after departure is allowed: $\delta_{ij} < f(i,j)$

# Reading in the data

In [234]:
import math
import numpy as np
import copy

# all fixed values:
alphaList = [0, 18, 35, 64, 35, 64, 0]
betaList  = [math.inf, 30, 45, 75, 50, 75, math.inf]
gammaList = [10, 10, 15, 28, 26, 32, 35, 43, 70, 30, 48, 72]
deltaList = [15, 25, 25, 35, 34, 60, 50, 50, 99, 80, 55, 80]
tTimeList = [10, 20, 25, 35, 35, 30, 15, 20, 30, 20, 25, 10]
nodes     = [1, 2, 3, 4, 5, 6, 7]
arcs      = {1:[2, 3, 5], 2:[4, 6, 7], 3:[2, 4], 4:[7], 5:[2, 6], 6:[7]}
feasible  = []

# to facilitate indexing, we zip the lists into dictionaries:
alpha = dict(zip(nodes, alphaList))
beta  = dict(zip(nodes, betaList ))
# gamma and delta have tuples as keys
arccount = 0
gamma = {}
delta = {}
tTime = {}
feasible = {}
for start in arcs.keys():
    for end in arcs[start]:
        gamma[(start, end)] = gammaList[arccount]
        delta[(start, end)] = deltaList[arccount]
        tTime[(start, end)] = tTimeList[arccount]
        feasible[(start, end)] = 0
        arccount += 1

# Finding a solution

In [235]:
def arrivalTime(pathInfo):
    """Writing a function to check for feasibility of arrival time and
    calculate the arrival time for some suggested extension
    of a current path, taking into account the current parking and travel times
    at all nodes in this path (without the extension). Here we assume that the
    arrival times given in the path are all feasible.
    
    pathinfo must be a dictionary with the keys path, parking and arrival.
    """
    endNode = pathInfo["path"][-1]
    print("Last node of current path:", endNode)
    if endNode == 7:
        movePath = tuple(pathInfo["path"])
        P[movePath] = copy.deepcopy(pathInfo)
        print("> Arrived at end node. Moved path to P.")
        return None
    # get all possible extension (arcs in A)
    nextNodes = arcs[endNode]
    print("> Current arrival time: ", pathInfo["arrival"][endNode])
    print("> Possible Extensions:", nextNodes)
    print(f"> Waiting time window at node {endNode}: [", alpha[endNode], ",",
        beta[endNode], "]", sep = "")
    # calculate new arrival times for all next nodes
    for nextNode in nextNodes:
        # getting the new arc
        currentArc = tuple([endNode, nextNode])
        # making a new entry in the dictionary, i.e. writing to the new path
        suggestedPath = copy.deepcopy(pathInfo)
        suggestedPath["path"].append(nextNode)
        currentPath = tuple(suggestedPath["path"])
        #print("  > pathInfo (this should not change):")
        #print("  ", pathInfo)
        print(f"  > Checking extension {nextNode}")
        #print("    Suggested Path is:")
        #print("  ", suggestedPath)
        print("  > Current path is:", currentPath)
        print("    > Departure time window: [", gamma[currentArc], ",",
              delta[currentArc], "]", sep = "")
        # arc to suggested next node is permanently blocked
        if pathInfo["arrival"][endNode] > delta[currentArc]:
            print("    Arrival at current end node occurs after departure time window",
                  f"to \n      node {nextNode} closes.")
        # arc to suggested next node is free, parking time not necessary
        elif gamma[currentArc] <= pathInfo["arrival"][endNode] <= delta[currentArc]:
            print(f"    > Arrival time at ({endNode}) is in the departure time window ",
                  f"([{gamma[currentArc]}, {delta[currentArc]}]). No parking time \n",
                  f"     at node {endNode} necessary.")
            suggestedPath["parking"][endNode] = [suggestedPath["arrival"][endNode],
                                                 suggestedPath["arrival"][endNode]]
            suggestedPath["arrival"][nextNode] = suggestedPath["arrival"][endNode] + \
                                                 tTime[currentArc]
            U[(currentPath)] = copy.deepcopy(suggestedPath)
        # arc to suggested node is not free, but parking on the end node allows
        # us to depart to the next node
        elif (alpha[endNode] <= pathInfo["arrival"][endNode] <= beta[endNode] and
                 gamma[currentArc] <= beta[endNode]):
            # we don't need the waiting time to end during the departure time window,
            # it is enough if the two windows overlap at all
            suggestedPath["parking"][endNode] = [suggestedPath["arrival"][endNode],
                                                gamma[currentArc]]
            suggestedPath["arrival"][nextNode] = gamma[currentArc] + \
                                                 suggestedPath["parking"][endNode][1]
            U[(currentPath)] = copy.deepcopy(suggestedPath)
            print("    > Parking is possible and parking time is ")
            print("     ", suggestedPath["parking"], ".", sep = "")
        # arc to suggested node is not free, and parking on the end node alone does not 
        # allow us to depart to the next node, but prolonging parking time at previous
        # nodes might allow us to depart to the next node
        elif pathInfo["arrival"][endNode] < gamma[currentArc]:
            print(f"    > Arrival at node {endNode} is too early to depart to node",
                  f"{nextNode}. Checking whether parking time can be increased",
                  f"at any previous node.")
            missingTime = gamma[currentArc] - pathInfo["arrival"][endNode]
            nodesChecked = [x for x in suggestedPath["path"] if x not in [endNode, nextNode]]
            print("    > Full path is", suggestedPath["path"])
            print("    > Previous nodes to be checked are")
            print("    ",  nodesChecked)
            for i, checkNode in enumerate(nodesChecked):
                checkDeparture = (checkNode, suggestedPath["path"][i+1])
                print(f"      > Checking arc {checkDeparture}")
                if alpha[checkNode] <= suggestedPath["arrival"][checkNode]:
                    print(f"      > Waiting at node {checkNode} is possible.")
                    oldParkingDuration = suggestedPath["parking"][checkNode][1] - suggestedPath["arrival"][checkNode]
                    #print("####", delta[checkDeparture])
                    #print("####", beta[checkNode])
                    #print("####", suggestedPath["parking"][checkNode][1] + missingTime)
                    newParkingEnd = min([delta[checkDeparture],
                                         beta[checkNode],
                                         suggestedPath["parking"][checkNode][1] + missingTime])
                    suggestedPath["parking"][checkNode] = [suggestedPath["arrival"][checkNode],
                           newParkingEnd]
                    print(f"      > Changing parking period at node {checkNode} to",
                          suggestedPath["parking"][checkNode])
                    missingTime = missingTime - suggestedPath["parking"][checkNode][1] - \
                                  suggestedPath["arrival"][checkNode] - oldParkingDuration
            print("######", suggestedPath)
            # recalculating all arrival times
            for i, node in enumerate(suggestedPath["path"]):
                if node == 1:
                    suggestedPath["arrival"][node] = 0
                else:
                    suggestedPath["arrival"][node] = \
                        gamma[(suggestedPath["path"][i-1], node)] + \
                        suggestedPath["parking"][node][1]
            U[(currentPath)] = copy.deepcopy(suggestedPath)
        else:
            print("    Current condition not implemented.")
    # remove fully checked path from U, move it to P
    movePath = tuple(pathInfo["path"])
    print("> Key of path to remove is:", movePath)
    P[movePath] = copy.deepcopy(pathInfo)

In [236]:
def selectNextPath(printing = True):
    paths = [path for path in U.keys()]
    selectedPath = paths[0]
    pathInfo     = copy.deepcopy(U[selectedPath])
    return(selectedPath)
    if(printing):
        print( "> Possible paths in U to choose from:")
        print(f"  {paths}")
        print( "> Selected Path:     ")
        print(f"  {selectedPath}")
        print("> Selected Path Data (pathInfo):")
        print(f"  {pathInfo}")

In [237]:
# Trying to put the thing into a loop
# initiate set of unprocessed paths with only the source:
U = {(1,):{"path":[1], "parking":{}, "arrival":{1:0}, "feasible":{}}} 
P = {}
print("> Current unprocessed paths:")
print(U)
print("> Set of processed paths:")
print(P)
print("")

while len(U) > 0:
    # select a path q from U and delete it from U
    nextPath = U[selectNextPath()]
    print("> Selecting a path:")
    print(" ", nextPath["path"])
    print("> Path looks like this:")
    print(" ", nextPath)
    print("> Removing this path from U.")
    del (U[selectNextPath()])
    arrivalTime(nextPath)
    print("> P")
    print(" ", P)
    print("> U")
    print(" ", U)
    print("\n\n")

> Current unprocessed paths:
{(1,): {'path': [1], 'parking': {}, 'arrival': {1: 0}, 'feasible': {}}}
> Set of processed paths:
{}

> Selecting a path:
  [1]
> Path looks like this:
  {'path': [1], 'parking': {}, 'arrival': {1: 0}, 'feasible': {}}
> Removing this path from U.
Last node of current path: 1
> Current arrival time:  0
> Possible Extensions: [2, 3, 5]
> Waiting time window at node 1: [0,inf]
  > Checking extension 2
  > Current path is: (1, 2)
    > Departure time window: [10,15]
    > Parking is possible and parking time is 
     {1: [0, 10]}.
  > Checking extension 3
  > Current path is: (1, 3)
    > Departure time window: [10,25]
    > Parking is possible and parking time is 
     {1: [0, 10]}.
  > Checking extension 5
  > Current path is: (1, 5)
    > Departure time window: [15,25]
    > Parking is possible and parking time is 
     {1: [0, 15]}.
> Key of path to remove is: (1,)
> P
  {(1,): {'path': [1], 'parking': {}, 'arrival': {1: 0}, 'feasible': {}}}
> U
  {(1, 2): 

KeyError: 2